In [49]:
from pybit.unified_trading import HTTP
from datetime import datetime,date,timedelta
import random
import string
import numpy as np

In [50]:
class BybitAPI:
    def __init__(self):
    
        self.session = HTTP(
            demo = True,
            testnet=False,
            api_key="mnOVAECL9uVKglkk6D",
            api_secret="HfFEJklZBhTHkTySnoCYxCm9Zs4xNH8SC0s1")

    def order(self,category, symbol, side, order_type, qty):

        orderLinkId = ''.join(random.choices(string.ascii_uppercase + string.digits, k=7))
        self.session.place_order(category = category,symbol = symbol,side = side,order_type = order_type,qty = qty,orderLinkId = orderLinkId)

    def get_option_delta(self):

        self.BTC_USDC_Value = float(self.session.get_tickers(category="spot",symbol="BTCUSDC")['result']['list'][0]['lastPrice'])

        strike_interval = 500
        atm_strike = round(self.BTC_USDC_Value / strike_interval) * strike_interval

        expiry = date.today() + timedelta(days=1)
        formatted_date = expiry.strftime('%d%b%y').upper()

        self.strike_ticker = f'BTC-{formatted_date}-{atm_strike}-C'
        option = self.session.get_tickers(category="option",symbol=self.strike_ticker)

        return option['result']['list'][0]['delta']

In [51]:
class grid_bot_trading(BybitAPI):
    
    def __init__(self):
        
        super().__init__()
        #amount of option to buy
        self.option_quantity = 0.2

    def execute(self):
        self.intialise_trade()
        self.intialize_grid()
    
    def intialise_trade(self):

        #get option delta
        self.delta = self.get_option_delta()

        #total amount of btc to short
        self.amount_btc = float(round((float(self.delta) * self.option_quantity),3))

        #intialise trade
        self.order("option",self.strike_ticker,"Buy","Market","0.2")
        self.order("linear","BTCPERP","Sell","Market",self.amount_btc)

        #average entry price
        position_info = self.session.get_positions(category="linear",symbol="BTCPERP")
        self.avg_entry_price = float(position_info['result']['list'][0]['avgPrice'])

    @staticmethod
    def calculate_mid_prices_range(grid_prices):
        mid_prices = [(grid_prices[i] + grid_prices[i + 1]) / 2 for i in range(1, len(grid_prices) - 1)]
        return mid_prices
    
    @staticmethod
    def divide_range(x, y, intervals=10):
        step = (y - x) / intervals
        grid_prices = [x + step * i for i in range(1, intervals + 1)]
        return grid_prices
    
    def intialize_grid(self):
        #number of grid levels 
        self.grid_levels = 10
        
        #btc value at which entire short position will be closed
        self.selloff_price = ((self.avg_entry_price*self.amount_btc)-(self.avg_entry_price*self.amount_btc*0.025))/self.amount_btc

        #amount of btc to be brought at each level
        self.qty_per_level = round((self.amount_btc / self.grid_levels),3)

        #grid prices to be sold off
        self.grid_sell_prices = grid_bot_trading.divide_range(self.avg_entry_price,self.selloff_price)
        # self.grid_buy_prices = grid_bot_trading.calculate_mid_prices_range(self.grid_sell_prices)

        request = []
        for i in self.grid_sell_prices:
            orderLinkId = ''.join(random.choices(string.ascii_uppercase + string.digits, k=7))

            request_1 = {
                "symbol": "BTCPERP",
                "side": "Buy",
                "orderType": "Limit",
                "qty": str(self.qty_per_level),
                "price": str(i),
                "orderLinkId": orderLinkId
                }
            
            request.append(request_1)

        self.session.place_batch_order(category="linear",request=request)

In [52]:
obj = grid_bot_trading()
obj.execute()

In [47]:
request = []
for i in obj.grid_sell_prices:
    orderLinkId = ''.join(random.choices(string.ascii_uppercase + string.digits, k=7))
    request_1 = {
        "symbol": "BTCPERP",
        "side": "Buy",
        "orderType": "Limit",
        "qty": str(2013),
        "price": str(i),
        "orderLinkId": orderLinkId
        }
    
    request.append(request_1)
print(request)

[{'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '60179.175', 'orderLinkId': 'QJF2SNB'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '60028.35', 'orderLinkId': 'SZ211FX'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59877.525', 'orderLinkId': 'XP38T38'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59726.700000000004', 'orderLinkId': 'RW259IW'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59575.875', 'orderLinkId': 'X494NH2'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59425.05', 'orderLinkId': 'JJP9HYH'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59274.225000000006', 'orderLinkId': 'NACQOCU'}, {'symbol': 'BTCPERP', 'side': 'Buy', 'orderType': 'Limit', 'qty': '2013', 'price': '59123.40000000001', 'orderLinkId': 'BW